In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import pandas as pd

In [ ]:
# configuracoes iniciais
url = "https://lista.mercadolivre.com.br/pecas/carros/injecao/injetores/reparo_DisplayType_LF_PriceRange_{inicial}-{final}"
options = Options()
options.headless = False
driver = webdriver.Firefox(options=options)
path_bases = "./Injetores-[Reparo]/bases/"

In [ ]:
# funcao pra pegar os links de uma pagina e salvar um csv
def get_urls_page(driver, num_page, i, j):
    lista_urls = []
    num_anuncios_aux = int(driver.find_element_by_xpath("/html/body/main/div/div[1]/aside/div[2]/span").text.split(" ")[0])
    num_anuncios = 50 if num_anuncios_aux > 50 else num_anuncios_aux
    
    for u in range(1, num_anuncios):
        xpath_url = "/html/body/main/div/div[1]/section/ol/li[{}]/div/div/div[2]/div[1]/a"
        xpath_url_mais_vendido = "/html/body/main/div/div[1]/section/ol/li[{}]/div/div/div[2]/div[2]/a"

        try:
            elem_url = driver.find_element_by_xpath(xpath_url.format(str(u)))
            url_anuncio = elem_url.get_attribute("href")
        except:
            elem_url = driver.find_element_by_xpath(xpath_url_mais_vendido.format(str(u)))
            url_anuncio = elem_url.get_attribute("href")

        lista_urls.append(lista_urls)
        
    df = pd.DataFrame({'urls':[lista_urls]})
    df_lista_precos.to_csv(path_bases + "df_lista_precos_" + str(i) + "_" + str(j) + "_" + "page_" + str(num_page) + ".csv")

In [ ]:
def iter_pages(driver, i, j):
    # identificar numero de paginas
    xpath_num_paginas = "/html/body/main/div/div[1]/section/div[3]/ul/li[2]"
    elem_num_paginas = driver.find_element_by_xpath(xpath_num_paginas)
    elem_num_paginas = int(elem_num_paginas.text.split(" ")[1])
    
    for num_page in range(1, int(elem_num_paginas)+1):
        get_urls_page(driver, num_page, i, j)
        time.sleep(2)
        if num_page == elem_num_paginas:
            pass
        else:
            driver.find_element_by_xpath("/html/body/main/div/div[1]/section/div[3]/ul/li[3]/a").click()

In [ ]:
iter_pages(driver, "1", "13")

In [ ]:
df_lista_precos = pd.read_csv("df_lista_precos.csv")
df_lista_precos = df_lista_precos[df_lista_precos["status"] != "OK"]

for index, row in df_lista_precos.iterrows():
    # abrir url
    i = str(row['precos'])
    f = str(int(i) + 1)
    url_get = url.format(
        inicial = i,
        final = f
    )
    driver.get(url_get)
    
    # verificar sem tem algum resultado
    try:
        elem_busca = driver.find_element_by_xpath("/html/body/main/div/div/div[2]/h3").text
    except:
        elem_busca = ""
    
    if elem_busca == 'Não há anúncios que correspondem à sua busca.':
        print(i + "-" + f + " - " + "Sem resultado")
    else:   
        iter_pages(driver, i, j)
        print(i + "-" + f + " - ", str(len(lista_urls)))
        
    lista_preco.pop(0)